# Owl
## A Retirement Planning Laboratory

This package is a retirement modeling framework for exploring the sensitivity of retirement financial decisions. Strictly speaking, it is not a planning tool, but more an environment for exploring *what if* scenarios. It provides different realizations of a financial strategy. One can certainly have a savings plan, but due to the volatility of financial investments, it is impossible to have a certain asset earnings plan. This does not mean one cannot make decisions. These decisions need to be guided with an understanding of the sensitivity of the parameters.This is exactly where this tool fits it. Given your savings and spending desires, it can generate different future realizations of your strategy under different market assumptions, helping to better understand your financial situation.

The algorithm in Owl is using the open-source HiGHS linear programming solver.
The complete formulation and detailed description of the underlying
mathematical model can be found
[here](https://raw.githubusercontent.com/mdlacasse/Owl/main/docs/owl.pdf).

Copyright &copy; 2024 - Martin-D. Lacasse

Disclaimers: *I am not a financial planner. You make your own decisions. This program comes with no guarantee. Use at your own risk.*

### <span style="color: blue"> README FIRST </span>
<span style="color:black;background:yellow;font-weight:bold">
Don't make changes directly to this file. Keep it as a working example. Therefore, it is recommended that you build your own case by making a copy of the template files provided where you will be able to enter your own numbers and explore your own assumptions.
</span>

# Tutorial 3 - Exploring beneficiary fractions using the case of Jack and Jill
This tutorial shows how to use advanced capabilities of Owl such as Monte Carlo simulations and running simulations over a historical range. We use both these capabilities to explore the effects of beneficiary fractions on *maxSpending* and *maxBequest* optimizations. We use a similar case as the one used in Tutorials 1 and 2 and introduce more of Owl's modeling capabilities. 

### Just some Python module bookkeeping
This command needs to be at the beginning of every Owl notebook.

In [ ]:
# If not installing owlplanner as a module
import sys
sys.path.insert(0, "../src")

In [ ]:
import owlplanner as owl

### Creating the plan (see tutorial 1 for details)
We first create a plan as we did in the *Tutorial 1*. We removed the comments for reducing the clutter.

In [ ]:
plan = owl.Plan(['Jack', 'Jill'], [1962, 1965], [89, 92], 'jack+jill-tutorial3', verbose=True)

In [ ]:
plan.setAccountBalances(
    taxable=[90.5, 60],
    taxDeferred=[600.2, 150],
    taxFree=[50 + 20.6, 40.8],
)

In [ ]:
plan.readContributions('../examples/jack+jill.xlsx')

In [ ]:
plan.setInterpolationMethod('s-curve')
plan.setAllocationRatios(
    'individual',
    generic=[[[60, 40, 0, 0], [70, 30, 0, 0]], [[60, 40, 0, 0], [70, 30, 0, 0]]],
)
# plan.showAllocations()

In [ ]:
plan.setPension([0, 10], [65, 65])
plan.setSocialSecurity([28, 25], [70, 70])

In [ ]:
plan.setSpendingProfile('smile', 60)
# plan.showProfile()

### Setting the heirs tax rate

In [ ]:
plan.setHeirsTaxRate(33)
plan.setLongTermCapitalTaxRate(15)

### Introducing partial spousal bequest
In some instances, the first spouse to pass can leave a fraction of her/his assets to the surviving spouse. Jack decides to give part of his assets to his children and leaves 25% of the value of his savings accounts to his surviving spouse. When not specified, default values are `[1, 1, 1]`. Changing the beneficiary fractions is done as follows. There is one value for each type of accounts: taxable, tax-deferred, and tax-free accounts.

In [ ]:
plan.setBeneficiaryFractions([0.25, 0.25, 0.25])
plan.setSpousalDepositFraction(0)

### Running a single case
We first run a single case with static *conservative* rates.
Jack and Jill desire to leave a final bequest of \\$200k (in today's \\$), leaving some security for Jill's last years. We also limit Roth conversions to a maximum of \\$100k for Jack and none for Jill, similar to the case we ran in Tutorials 1 and 2.

In [ ]:
plan.setRates('conservative')
options = {'maxRothConversion': 100, 'noRothConversions': 'Jill'}
# options['solver'] = 'MOSEK'
options['bequest'] = 200
plan.solve('maxSpending', options);
plan.summary()

Looking at savings accounts, notice the drop in value taking place as Jack splits his bequest between his spouse and his children in 2051.

In [ ]:
plan.showAccounts()

### Running Monte Carlo simulations
Before running Monte Carlo simulations, we need to configure a stochastic rate-generating method as explained in Tutorial 2. We will use a relatively conservative return projection of 8% +/- 17% for the S&P 500 and 5% +/ 8% for bonds, with 3% +/- 2% for the inflation. We use correlations derived from historical values.

In [ ]:
# Mean returns
my_means = [8, 4, 4, 3]
# Volatility
my_stdev = [17, 8, 8, 2]
# Correlations matrix between rates of return of different assets
# my_corr = [[1, 0.46, 0.06, -.12], [0.46, 1, 0.68, -.27], [0.06, 0.68, 1, -.21], [-.12, -.27, -.21, 1]]
# These are only the off-diagonal elements
offdiag_corr = [.46, .06, -.12, .68, -.27, -.21]
plan.setRates('stochastic', values=my_means, stdev=my_stdev, corr=offdiag_corr)

In [ ]:
# Display a single instance resulting from this choice.
# plan.showRatesCorrelations(shareRange=False)
# plan.showRates()

### Preparing the case to run
We now run many of those cases using the `runMC()` method. We repeat here the options used above for completeness.

When setting beneficiary fractions to less than unity, the `runMC()` and `runHistoricalRange()` methods will also report a histogram of the after-tax value of the non-spousal part of the bequest made at the passing of the first spouse.

What are these cases telling us? Let's look at the specific example at hand.

In [ ]:
options = {'maxRothConversion': 100, 'noRothConversions': 'Jill'}
options['solver'] = 'MOSEK'

In [ ]:
options['bequest'] = 200
plan.runMC('maxSpending', options, 500);

*Partial* is the post-tax partial bequest left to individuals others than Jill at the passing of Jack, in that case, Jack's children. The range of values obtained in solutions is also reported as well as the number of solutions with 0 as an answer. The amount that Jack leaves to his children can vary from \\$0 to about \\$1 M, and shows the largest variability, explaining the large difference between the median and the average. Mean values are indicated by $\bar{x}$. What strikes here the the number of solutions which have Jack's account depleted. We will discuss these cases.

Almost all cases are successful at leaving a \\$200k final bequest, but the net spending required for achieving this constraint can lead to small values in some scenarios. However, the median value $M$ for net spending is about \\$88k, very close to the \\$90k desired by Jack and Jill. Recall that we are using a more conservative return of 8%, with a volatility of 17% for the S&P 500. It is therefore natural that the number we find here is slightly lower than what the historical returns of the market have been observed to provide. Some of these scenarios will yield more than the median, but some others will require a net spending of less than \\$50k. All these scenarios make a probability of close to 100\% for leaving a \\$200k bequest at Jill's passing as requested. This is in addition to a median of about \\$20k left at Jack's passing. We can see that the mean value left by Jack to his children is around \\$130k while the median is much lower, due to many scenarios depleting Jack's account at his passing. What is to note is that several scenarios deplete Jack's account to reduce the partial bequest in view of maximizing the net spending. This is the action of the optimization working to maximize its objective function, which  is the net spending in this particular case. That is, the optimizer will select scenarios where the net spending will be increased at the expense of decreasing the partial bequest. What can potentially still hold money in Jack's account and allow to provide a partial bequest is the restriction imposed by satisfying a certain spending profile which limits the depletion of the accounts. That is, more gains can be realized in certain scenarios that the ability to spend.

The probability of success might be misleading here as this is the number of cases for which a solution was found, regardless of the optimal value for a net spending basis. A more useful information would consist of the fraction of all solutions larger than a value $x$. This value can be derived from the histogram on the right. This would give an estimate of the probability of having a net spending larger than $x$ conditional to delivering a final bequest of \\$200k.

Alternatively, one can look at the maximum bequest left at Jill's passing under the constraint of a net spending starting at \$80k this year (obviously in today's money). This is what the next case will do.

In [ ]:
options['netSpending'] = 80
plan.runMC('maxBequest', options, 500);

In this case, a relatively good fraction of scenarios are successful in providing the desired \\$80k net spending amount modulated over the duration of the plan. The median bequest left at the end of the plan is close to \\$500k, while Jack's partial bequest is only a little less. A success rate larger than 85% is considered relatively good, especially considering the fact that Jack and Jill are always able to re-adjust their net spending plan as other forward-looking estimates are performed in the future, and that the partial bequest left to Jack's children is a percentage, not an absolute quantity. This suggests that by reducing their spending by \\$10k, Jack and Jill can provide a total bequest of about \\$1 M in two separate bequests, a first in 2051, and then another one in 2057.

### Running cases over a historical range or years
As with full spousal asset transfer, we can also run the same cases using historical data over a range of years and compile the outcomes in a histogram. For this purpose, there is no need to pre-select specific rates using the `setRates()` method as the choice of *historical* rates is implicit. Only the year range is required.

Let's look at a specific example. For optimizing the net spending amount, we specify *maxSpending* in the following call to optimize the plan over the 63 years following 1928:

In [ ]:
plan.runHistoricalRange('maxSpending', options, 1928, 1990);

When considering more historical data than a single year, we find that most solutions lead to empty accounts at Jack's passing. 

Let's now consider the case for *maxBequest* under a desired net spending.

In [ ]:
plan.runHistoricalRange('maxBequest', options, 1928, 1990);

The resulting success rate when considering all historical data is 100% for a desired net spending of \\$80k. This is also not surprising as the original estimates for the net spending and the bequest were derived from running a case in 1969, the second next worst year of historical sequences of returns, and were slightly more than \\$80k.